In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# 2018-09-11_Getting started with BRIAN

# 1/Neurons

In [2]:
from brian2 import *

ModuleNotFoundError: No module named 'brian2'

In [ ]:
#units demo
#micro from shortcuts
print(-60e-3*volt)
print(-60*mvolt)
print(120*amp)
print(120*mamp)
#type multiplication
print(120*mamp*60*ohm)

## Equation demos

In [ ]:
#clean previous variables
start_scope()

#simple neuron
tau = 10*ms

equations = '''
dv/dt = (1-v)/tau : 1
'''

#a group of one neuron using the previous equation
pop = NeuronGroup(1, equations, method='exact')
monitor = StateMonitor(pop, 'v', record = True)

#run the sim
run(50*ms)

#plot the sim
plot(monitor.t/ms, monitor.v[0])
xlabel('Time in ms')
ylabel('Voltage of neuron')

In [ ]:
#oscillating behavior
start_scope()

tau = 10*ms
equations = '''
dv/dt = (sin(2*pi*250*Hz*t)-v)/tau : 1
'''

pop = NeuronGroup(1, equations, method = 'euler')
monitor = StateMonitor(pop, 'v', record = True)

#init
pop.v = 2

run(100*ms)

plot(monitor.t/ms, monitor.v[0])
xlabel('Time in ms')
ylabel('Voltage of neuron')

## Spiking demos

In [ ]:
#with spikes, much slower than a simple equation when the neuron has to behave too

start_scope()

tau = 20*ms
equations = '''
dv/dt = (1-v)/tau : 1
'''

pop = NeuronGroup(1, equations, threshold='v>.8', reset = 'v = 0', method = 'euler')
monitor = StateMonitor(pop, 'v', record = True)
spikemonitor = SpikeMonitor(pop)

pop.v = .5

run(100*ms)

plot(monitor.t/ms, monitor.v[0], label = 'voltage variation')
for spikes in spikemonitor.t :
    axvline(spikes/ms, ls = '-.',color = 'red', label = 'spike')
xlabel('Time in ms')
ylabel('Voltage of neuron')

print('Spikes at : %s' % spikemonitor.t[:])

In [ ]:
#with spikes and refraction after firing (dont forget to mention it in the equations)

start_scope()

tau = 20*ms
equations = '''
dv/dt = (1-v)/tau : 1 (unless refractory)
'''

pop = NeuronGroup(1, equations, threshold='v>.8', refractory=10*ms, reset = 'v = 0', method = 'euler')
monitor = StateMonitor(pop, 'v', record = True)
spikemonitor = SpikeMonitor(pop)

pop.v = .5

run(100*ms)

plot(monitor.t/ms, monitor.v[0], label = 'voltage variation')
for spikes in spikemonitor.t :
    axvline(spikes/ms, ls = '-.',color = 'red', label = 'spike')
xlabel('Time in ms')
ylabel('Voltage of neuron')

print('Spikes at : %s' % spikemonitor.t[:])

In [ ]:
#spiking behavior of a group of neuron during time
#rasterplot is the word I was looking for

#with spikes and refraction after firing (dont forget to mention it in the equations)
start_scope()

N=30
tau = 10*ms
equations = '''
dv/dt = (1-v)/tau : 1 (unless refractory)
'''

pop = NeuronGroup(N, equations, threshold='v>.8', refractory=5*ms, reset = 'v = 0', method = 'euler')
monitor = StateMonitor(pop, 'v', record = True)
spikemonitor = SpikeMonitor(pop)

pop.v = 'rand()' #otherwise it's too uniform

run(100*ms)

plot(spikemonitor.t/ms, spikemonitor.i, '.k') #.k otherwise it's a line plot

xlabel('Time in ms')
ylabel('Neuron index')

print('Spikes at : %s' % spikemonitor.t[:])

## Neuron-wise parameters demo

In [ ]:
#without noise
start_scope()

N = 30
tau = 10*ms
v0_max = 3.

#v0 is a per neuron parameter with unit one (dimensionless)
equations = '''
dv/dt = (v0-v)/tau : 1 (unless refractory)
v0 : 1 
'''

population = NeuronGroup(N, equations,
                         threshold = 'v>.8', reset = 'v=0', refractory = 2*ms, method = 'exact')
spikemonitor = SpikeMonitor(population)

#each neuron's v0 is then initialised to a value between 0 to v_max (i is the index)
#the greater the index the greater v0, the greater the firing rate
population.v0 = 'i*v0_max/(N-1)'

run(100*ms)

figure(figsize = (15,8))
subplot(121)
plot(spikemonitor.t/ms, spikemonitor.i, '.k')
xlabel('Time in ms')
ylabel('Neuron index')
title('Raster plot of the population')
subplot(122)
plot(population.v0, spikemonitor.count/100*ms) #spikemonitor.count is an array of the number of spikes per neuron
xlabel('v0')
ylabel('Firing rate')
title('Input vs Firing rate (If curve)')

In [ ]:
#with noise, using xi
start_scope()

N = 30
tau = 10*ms
v0_max = 3.
sigma = .2

#v0 is a per neuron parameter with unit one (dimensionless)
equations = '''
dv/dt = (v0-v)/tau+sigma*xi*tau**-.5 : 1 (unless refractory)
v0 : 1 
'''

population = NeuronGroup(N, equations,
                         threshold = 'v>.8', reset = 'v=0', refractory = 5*ms, method = 'euler')
spikemonitor = SpikeMonitor(population)

#each neuron's v0 is then initialised to a value between 0 to v_max (i is the index)
#the greater the index the greater v0, the greater the firing rate
population.v0 = 'i*v0_max/(N-1)'

run(1000*ms)

figure(figsize = (15,8))
subplot(121)
plot(spikemonitor.t/ms, spikemonitor.i, '.k')
xlabel('Time in ms')
ylabel('Neuron index')
title('Raster plot of the population')
subplot(122)
plot(population.v0, spikemonitor.count/1000*ms) #spikemonitor.count is an array of the number of spikes per neuron
xlabel('v0')
ylabel('Firing rate')
title('Input vs Firing rate (If curve)')

## End of the Neuron tutorial

In [ ]:
import random 

#with equations to describe behavior instead of a fixed limit
start_scope()

N=30
tau = 10*ms
resting_potential = -65*mV

firing_threshold = -50*mV
delta_vt0 = 5*mV

tau_t = 10*ms
sigma = 0.5*(firing_threshold-resting_potential)
v_drive = 2*(firing_threshold-resting_potential)

duration = 50*ms
random_index = random.randint(0,N)-1
equations = '''
dv/dt = (v_drive+resting_potential-v)/tau + sigma*xi*tau**-.5 : volt
dvt/dt = (firing_threshold-vt)/tau_t : volt
'''

reset = '''
v = resting_potential
vt += delta_vt0
'''

#notice the string casting from the reset equation into the treshold one just below
population = NeuronGroup(N, equations, threshold = 'v>vt', reset = reset, refractory = 5*ms, method = 'euler') 
spikemonitor = SpikeMonitor(population)

monitor = StateMonitor(population, 'v', record = random_index)

population.v = 'rand()*(firing_threshold-resting_potential)+resting_potential'
population.vt = vt0

run(duration)

figure(figsize=(15,8))
subplot(121)
__ = hist(spikemonitor.t/ms, 100, histtype='stepfilled', facecolor = 'k',
        weights = ones(len(spikemonitor))/(N*defaultclock.dt))
xlabel('Time in ms')
ylabel('Total firing rate sps/s')
title('Population FR over time')

subplot(122)

plot(monitor.t/ms, monitor.v[0])
#we get only the spiketimes for the recorded neuron using monitor.spike_trains() dictionnary
for spikes in spikemonitor.spike_trains()[random_index]:
    axvline(spikes/ms, ls = '-.',color = 'red', label = 'spike')
axhline(-50e-3, color = 'orange', label = 'Firing Threshold')
xlabel('Time in ms')
ylabel('Voltage')
legend()
title('Voltage variation from one random neuron')

# 2/ Synapses

In [ ]:
from brian2 import *

In [ ]:
#previously neuron were synchronized by system variable, now we'll use synapses
#god it's slow
start_scope()

equations = '''
dv/dt = (I-v)/tau : 1
I : 1
tau : second
'''

population = NeuronGroup(2, equations, threshold = 'v>1', reset = 'v=0', method = 'euler')

#neuron one spikes fairly fast but neuron two won't spike on its own
population.I = [2,0]
population.tau = [10, 100]*ms

#when the spike occurs on_pre neuron, the post synaptic neuron gains .2V (this is a shomewhat weight)
synapses = Synapses(population, population, on_pre = 'v_post += .2',
                   delay = 5*ms)

#0 and 1 are ID, not weights
synapses.connect(i=0, j=1)

monitor = StateMonitor(population, 'v', record = True)

run(100*ms)

figure(figsize=(12,8))
plot(monitor.t/ms, monitor.v[0], label = 'Neuron 1')
plot(monitor.t/ms, monitor.v[1], label = 'Neuron 2')

axvline(6.8, color = 'black', label = 'Synaptic cleft delay')
axvline(11.8, color = 'black')

xlabel('Time in ms')
ylabel('Voltage')
legend()

In [ ]:
#with dynamic synapse weight
#toy example where the ID of a synapse increase its threshold as well as its delay
start_scope()

equations = '''
dv/dt = (I-v)/tau : 1
I : 1
tau : second
'''

population = NeuronGroup(3, equations, threshold = 'v>1', reset = 'v=0', method = 'euler')

#three neurons, only the first one spikes
population.I = [2,0,0]
population.tau = [10, 100, 100]*ms

#when the spike occurs on_pre neuron, the post synaptic neuron gains .2V (this is a shomewhat weight)
synapses = Synapses(population, population, 'w : 1', on_pre = 'v_post += w')

#0 and 1 are ID, not weights
synapses.connect(i=0, j=[1,2])
synapses.w = 'j*.2'
synapses.delay = 'j*.2*ms' #don't forget the ms

monitor = StateMonitor(population, 'v', record = True)

run(100*ms)

figure(figsize=(12,8))
plot(monitor.t/ms, monitor.v[0], label = 'Neuron 1')
plot(monitor.t/ms, monitor.v[1], label = 'Neuron 2')
plot(monitor.t/ms, monitor.v[2], label = 'Neuron 3')

'''
axvline(6.8, color = 'black', label = 'Synaptic cleft delay')
axvline(11.8, color = 'black')
'''

xlabel('Time in ms')
ylabel('Voltage')
legend()

In [ ]:
#probabilistic connectivity
# connects i with j as long as it's not a neuron-neuron autapse, with a probability of .2
    
#cool fonction from the demo
def visualise_connectivity(S):
    Ns = len(S.source)
    Nt = len(S.target)
    
    figure(figsize=(10, 4))
    subplot(121)
    plot(zeros(Ns), arange(Ns), 'ok', ms=10, color = 'blue') #pre
    plot(ones(Nt), arange(Nt), 'ok', ms=10, color = 'red') #post
    for i, j in zip(S.i, S.j): #line plotting from pre to post
        plot([0, 1], [i, j], '-k')
    #graphicals
    xticks([0, 1], ['Source', 'Target'])
    ylabel('Neuron index')
    xlim(-0.1, 1.1)
    ylim(-1, max(Ns, Nt))
    
    subplot(122)
    plot(S.i, S.j, 'ok')
    xlim(-1, Ns)
    ylim(-1, Nt)
    xlabel('Source neuron index')
    ylabel('Target neuron index')

start_scope()
N=10
pop = NeuronGroup(N, 'v:1')

for probas in [0.1, .2, .5, .7, 1] :
    syn = Synapses(pop, pop)
    syn.connect(condition ='i!=j', p = probas)
    visualise_connectivity(syn)

In [ ]:
#neighbouring connectivity, iaoi neurons are 4 units close and excluding autapses

start_scope()
N=10
pop = NeuronGroup(N, 'v : 1')
syn = Synapses(pop, pop)
syn.connect(condition='abs(i-j)<4 and i!=j', skip_if_invalid = True)
visualise_connectivity(syn)

In [ ]:
#stop ! STDP time

import warnings
warnings.filterwarnings("ignore") #diss useless dot product deperecation vars

start_scope()

tau_pre = tau_post = 20*ms
wmax = 0.01

#traces of the synaptic activity
Apre = 0.01 
Apost = -Apre*tau_pre/tau_post*1.05

pop = NeuronGroup(2, 'v:1', threshold='t>(1+i)*10*ms', refractory = 100*ms, method='exact')

'''Three dynamic variables, w, onpre, onpost
Here we use clock driven updates but we could use
event driven meaning the variable is only updated after a spike (otherwise it's not efficient)
on_pre : first line updates the weight, second line updates the synaptic activity, third lines clips the weight
on_post : same as on_pre but doesn't update synaptic weight
'''
syn = Synapses(pop, pop,
              '''
              w : 1
              dapre/dt = -apre/tau_pre : 1 (clock-driven)
              dapost/dt = -apost/tau_post : 1 (clock-driven)
              ''',
              on_pre='''
              v_post +=w
              apre += Apre
              w = clip(w+apost, 0, wmax)
              ''',
              on_post='''
              apost+=Apost
              w=clip(w+apre, 0, wmax)
              ''',
                method='exact')
syn.connect(i=0, j=1)
monitor = StateMonitor(syn, ['w', 'apre', 'apost'], record = True)

run(30*ms)

figure(figsize=(12,8))
subplot(121)
plot(monitor.t/ms, monitor.apre[0], label = 'Presyn activity')
plot(monitor.t/ms, monitor.apost[0], label = 'Postsyn activity')
legend()
subplot(122)
plot(monitor.t/ms, monitor.w[0], label = 'Weight')

# 3/ Simulation

In [ ]:
# an example of poisson neurons spiking to a leaky fire and integrate neuron
# how does the firing rate of this neuron varies depending on its time constant tau ?
#we'll use the store and restore to save up computation time, which can also be a good idea for training/testing

start_scope()

num_inputs = 100
input_rate = 10 * Hz
weight = .1

tau_range = linspace(1,10, 30)*ms
output_rates = []

poisson_pop = PoissonGroup(num_inputs, rates = input_rate)

equations = '''
dv/dt = -v/tau : 1
'''
population = NeuronGroup(1, equations, threshold ='v>1', reset = 'v=0', method = 'exact')
syn = Synapses(poisson_pop, population, on_pre ='v+=weight')
syn.connect()

spikemonitor = SpikeMonitor(population)
store() #we can save the state of the network and use a restore loop to save computation time

for tau in tau_range :
    restore() #here we go
    run(300*ms)
    output_rates.append(spikemonitor.num_spikes/second)
    
plot(tau_range/ms, output_rates)
xlabel('Time constant Tau')
ylabel('Firing rate avg over 300 ms')

In [ ]:
#dynamic variable change for a HH neuron, we'll change the amplitude of a current injected
#will be interesting for delay learnings 
#we could also do neat tricks to change ion concentrations
# VERSION WORKING ON STANDALONE C++ CODE

start_scope()
# Parameters
area = 20000*umetre**2
Cm = 1*ufarad*cm**-2 * area
gl = 5e-5*siemens*cm**-2 * area

#electrochemical forces
El = -65*mV
EK = -90*mV
ENa = 50*mV

#chemical gradients
g_na = 100*msiemens*cm**-2 * area
g_kd = 30*msiemens*cm**-2 * area
VT = -63*mV

# The Nobel-worth equations
eqs_HH = '''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I)/Cm : volt
dm/dt = 0.32*(mV**-1)*(13.*mV-v+VT)/
    (exp((13.*mV-v+VT)/(4.*mV))-1.)/ms*(1-m)-0.28*(mV**-1)*(v-VT-40.*mV)/
    (exp((v-VT-40.*mV)/(5.*mV))-1.)/ms*m : 1
dn/dt = 0.032*(mV**-1)*(15.*mV-v+VT)/
    (exp((15.*mV-v+VT)/(5.*mV))-1.)/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
I : amp
'''
group = NeuronGroup(1, eqs_HH,
                    threshold='v > -40*mV',
                    refractory='v > -40*mV',
                    method='exponential_euler')
group.v = El #init voltage at resting potential

statemon = StateMonitor(group, 'v', record=True)
spikemon = SpikeMonitor(group, variables='v')

#instead of looping and changing the value of I with group.I, we can use group.run_regularly for massive speedups
#the code has to be specific to the neurongroup /!/
group.run_regularly('I = rand()*50*nA', dt = 10*ms)
run(50*ms)

figure(figsize=(12, 8))
for l in range(5): #five runs
    axvline(l*10, ls='--', c='k', label = 'Var change')
    
axhline(El/mV, ls='-', c='lightgray', lw=3, label = 'Resting Potential')

plot(statemon.t/ms, statemon.v[0]/mV, '-b')
plot(spikemon.t/ms, spikemon.v/mV, 'ob')
xlabel('Time (ms)')
ylabel('v (mV)')

In [ ]:
#dynamic variable change for a HH neuron, we'll change the amplitude of a current injected
#will be interesting for delay learnings
# VERSION NOT WORKING ON STANDALONE C++ CODE
#this method looks the fastest

start_scope()
# Parameters
area = 20000*umetre**2
Cm = 1*ufarad*cm**-2 * area
gl = 5e-5*siemens*cm**-2 * area

#electrochemical forces
El = -65*mV
EK = -90*mV
ENa = 50*mV

#chemical gradients
g_na = 100*msiemens*cm**-2 * area
g_kd = 30*msiemens*cm**-2 * area
VT = -63*mV

# The Nobel-worth equations
eqs_HH = '''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I)/Cm : volt
dm/dt = 0.32*(mV**-1)*(13.*mV-v+VT)/
    (exp((13.*mV-v+VT)/(4.*mV))-1.)/ms*(1-m)-0.28*(mV**-1)*(v-VT-40.*mV)/
    (exp((v-VT-40.*mV)/(5.*mV))-1.)/ms*m : 1
dn/dt = 0.032*(mV**-1)*(15.*mV-v+VT)/
    (exp((15.*mV-v+VT)/(5.*mV))-1.)/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
I : amp
'''
group = NeuronGroup(1, eqs_HH,
                    threshold='v > -40*mV',
                    refractory='v > -40*mV',
                    method='exponential_euler')
group.v = El #init voltage at resting potential

statemon = StateMonitor(group, 'v', record=True)
spikemon = SpikeMonitor(group, variables='v')

#We can also use network operation static methods, which support any python operation (yei)
#Won't work on C++ tree generation though (nei)

@network_operation(dt=10*ms)
def change_I():
    group.I = rand()*50*nA
run(50*ms)

figure(figsize=(12, 8))
for l in range(5): #five runs
    axvline(l*10, ls='--', c='k', label = 'Var change')
    
axhline(El/mV, ls='-', c='lightgray', lw=3, label = 'Resting Potential')

plot(statemon.t/ms, statemon.v[0]/mV, '-b')
plot(spikemon.t/ms, spikemon.v/mV, 'ob')
xlabel('Time (ms)')
ylabel('v (mV)')

In [ ]:
#shared variable exemple, just in case we'd want to synchronize some variables

start_scope()
N = 3
eqs_HH_3 = '''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I)/C : volt
dm/dt = 0.32*(mV**-1)*(13.*mV-v+VT)/
    (exp((13.*mV-v+VT)/(4.*mV))-1.)/ms*(1-m)-0.28*(mV**-1)*(v-VT-40.*mV)/
    (exp((v-VT-40.*mV)/(5.*mV))-1.)/ms*m : 1
dn/dt = 0.032*(mV**-1)*(15.*mV-v+VT)/
    (exp((15.*mV-v+VT)/(5.*mV))-1.)/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
I : amp (shared) # everything is the same except we've added this shared
C : farad
'''
group = NeuronGroup(N, eqs_HH_3,
                    threshold='v > -40*mV',
                    refractory='v > -40*mV',
                    method='exponential_euler')
group.v = El
group.C = array([0.8, 1, 1.2])*ufarad*cm**-2*area
statemon = StateMonitor(group, 'v', record=True)
group.run_regularly('I = rand()*50*nA', dt=10*ms)
run(50*ms)
figure(figsize=(9, 4))
for l in range(5):
    axvline(l*10, ls='--', c='k')
axhline(El/mV, ls='-', c='lightgray', lw=3)
plot(statemon.t/ms, statemon.v.T/mV, '-')
xlabel('Time (ms)')
ylabel('v (mV)');

In [ ]:
#image input, a wee bit of fun
from matplotlib.image import imread
start_scope()
img = (1-imread('./misc/PDS_example.png'))[::-1, :, 0]

num_samples, N = img.shape #x,y length of the img, will be used for rasterplotting dims

ta = TimedArray(img, dt=1*ms) #dummy time array to display as a rasterplot
A = 1.5
tau = 2*ms

#the term before xi can be used as a knob that modifies the noise quantity
equations = '''
dv/dt = (A*ta(t,i)-v)/tau+0.5*xi*tau**-.5 : 1
'''
pop = NeuronGroup(N, equations, threshold='v>1', reset = 'v=0', method='euler')
spikemonitor = SpikeMonitor(pop)

run(num_samples*ms)
figure(figsize = (1.86*10,10))
plot(spikemonitor.t/ms, spikemonitor.i, '.k', ms = 3)
xlim(0, num_samples)
ylim(0, N)
xlabel('Temps en milisecondes')
ylabel('Evenement neuronal')
title("Ca c'est de l'Optimal Packing qui envoie !")
savefig('./misc/OptimalToy.png')